# Demo query FAIR Package Statement

This notebook demonstrates how to programmatically query the FAIR Package Statement registered in a given FAIR Data Point (FDP)

# 0. Install Required Libraries

In [ ]:
%pip install rdflib pandas

## 1. Import Required Libraries

We will import the necessary Python libraries for file and notebook manipulation.

In [ ]:
# Import required libraries
from rdflib import Graph
from pandas import DataFrame
from urllib.parse import unquote

# 2. Get Package Statement from FAIR data station

This example demonstrates how to query a Data Station (local GraphDB triplestore, repository: `package-statements`) using RDFLib

In [ ]:
# Get MammaOnco RDF data from URL referenced in FDP:
mammaonco_rdf = "https://graph-zin.dci.thehyve.nl/repositories/mamma-onco/statements"

g = Graph()
g.parse(mammaonco_rdf)
print(f"Number of triples in the graph: {len(g)}")

# 3. What is/are the Intervention(s)

In [ ]:
query = """
PREFIX dcterms: <http://purl.org/dc/terms/>
PREFIX fpr: <https://w3id.org/zinl/fpr-o#>

SELECT DISTINCT ?title ?iic
{
    ?iic a fpr:IICAssessment ;
    	fpr:hasIntervention ?intervention .
    ?intervention dcterms:title ?title .
} LIMIT 100
"""
# Execute the query and print results
results = g.query(query)
df = DataFrame(results.bindings, columns=results.vars)
df = DataFrame(results.bindings, columns=results.vars)
df.apply(lambda x: x.str.replace("https://w3id.org/zinl/package-statements/", "data:"))


# 4. What is the Indication (PICO)?

In [ ]:

query = """
PREFIX pico: <http://data.cochrane.org/ontologies/pico/>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX fpr: <https://w3id.org/zinl/fpr-o#>

SELECT DISTINCT ?title ?age ?sex ?condition
{
    ?iic a fpr:IICAssessment ;
    	fpr:hasIntervention ?intervention ;
        fpr:hasIndication ?population .
    ?intervention dcterms:title ?title .
    ?population pico:sex ?sex ;
    	pico:age ?age ;
    	pico:condition ?condition .
} LIMIT 100
"""

# Execute the query and print results
results = g.query(query)
df = DataFrame(results.bindings, columns=results.vars)
df = df.apply(lambda x: x.str.replace("https://w3id.org/zinl/fpr-tax#", "tax:"))
df.apply(lambda x: x.str.replace("%20", " "))

# 5. Trend assumption: how many patients fall within eligble indication?



In [ ]:
query = """
PREFIX qb: <http://purl.org/linked-data/cube#>
PREFIX fpr: <https://w3id.org/zinl/fpr-o#>

SELECT ?iic ?bia ?nrOfPatients ?timepoint
{
    ?iic a fpr:IICAssessment ;
    	fpr:hasBIA ?bia .
    ?bia fpr:hasTrendAssumption ?trendAssumption .
    ?obs qb:dataSet ?trendAssumption .
    ?obs fpr:numberOfPatients ?nrOfPatients ;
    	fpr:timepoint ?timepoint .
} LIMIT 100
"""
# Execute the query and print results
results = g.query(query)
df = DataFrame(results.bindings, columns=results.vars)
df.apply(lambda x: x.str.replace("https://w3id.org/zinl/package-statements/", "data:"))

# 6. How many patients were estimated to use this Intervention?




In [ ]:
query = """
PREFIX qb: <http://purl.org/linked-data/cube#>
PREFIX fpr: <https://w3id.org/zinl/fpr-o#>

SELECT DISTINCT ?intervention ?costType ?childIntervention ?nrOfPatients ?costs ?timepoint
{
    ?iic a fpr:IICAssessment ;
    	fpr:hasBIA ?bia ;
    	fpr:hasIntervention ?intervention .
    ?bia fpr:hasCostEstimation ?costEstimation .
    ?obs qb:dataSet ?costEstimation ;
    	fpr:costType ?costType ;
    	fpr:hasIntervention ?childIntervention ;
    	fpr:hasNumberOfPatients ?nrOfPatients ;
			fpr:hasTotalCosts ?costs ;
    	fpr:hasTimepoint ?timepoint .
}
ORDER BY ?intervention ?costType
LIMIT 100
"""
# Execute the query and print results
results = g.query(query)
df = DataFrame(results.bindings, columns=results.vars)
df.apply(lambda x: x.str.replace("https://w3id.org/zinl/package-statements/", "data:"))

